In [ ]:
from transformers import pipeline
from transformers import CLIPProcessor, CLIPModel 

In [ ]:
checkpoint = "openai/clip-vit-base-patch16"

model = CLIPModel.from_pretrained(checkpoint)
processor = CLIPProcessor.from_pretrained(checkpoint)

In [ ]:
!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar
!tar -xvf VOCtrainval_11-May-2012.tar

In [ ]:
from datasets import load_dataset

dataset = load_dataset("imagefolder", data_dir="./VOCdevkit/VOC2012/JPEGImages")

In [ ]:
num_samples = 5
seed = 221
candidate_subset = dataset["train"].shuffle(seed=seed).select(range(num_samples))

In [ ]:
candidate_subset['image'][3]

In [ ]:
for i, example in enumerate(candidate_subset):
    inputs = processor(images=example['image'], text=['baby', 'car', 'men', 'bus'],  return_tensors="pt", padding=True)
    outputs = model(**inputs)
    logits = outputs.logits_per_image[0]
    probs = logits.softmax(dim=-1).detach().numpy()
    scores = probs.tolist()

    result = [
    {"score": score, "label": candidate_label}
    for score, candidate_label in sorted(zip(probs, ['baby', 'car', 'men', 'bus']), key=lambda x: -x[0])]

    print(result)